# Training model for Machine Translation

Library Imports for the jupyter notebook. We are refering to this [blog](https://medium.com/@hunter-j-phillips/putting-it-all-together-the-implemented-transformer-bfb11ac1ddfehttps://medium.com/@hunter-j-phillips/putting-it-all-together-the-implemented-transformer-bfb11ac1ddfe) to understand attention network in depth

In [1]:
!pip install -q portalocker

# importing required libraries
import math
import copy
import time
import random
import spacy
import numpy as np
import os 

# torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import torch.optim as optim

# load and build datasets
import torchtext
from torchtext.data.functional import to_map_style_dataset
from torch.nn.functional import pad
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import portalocker

# visualization packages
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


device(type='cuda')

# Reading the dataframe and converting it into iterable for consuming in pytorch dataset

In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
df = pd.read_csv("/kaggle/input/machine-translation-dataset-de-en/translation_train.csv")
df.head()

,english,german
0,"Two young, White males are outside near many b...",Zwei junge weiße Männer sind im Freien in der ...
1,Several men in hard hats are operating a giant...,Mehrere Männer mit Schutzhelmen bedienen ein A...
2,A little girl climbing into a wooden playhouse.,Ein kleines Mädchen klettert in ein Spielhaus ...
3,A man in a blue shirt is standing on a ladder ...,Ein Mann in einem blauen Hemd steht auf einer ...
4,Two men are at the stove preparing food.,Zwei Männer stehen am Herd und bereiten Essen zu.


In [4]:
df.shape

(29000, 2)

In [5]:
# Perform train - val split
train_df=df.sample(frac=0.95,random_state=200)
val_df=df.drop(train_df.index)

In [6]:
def create_iterable(sample_df):
    """
    This converts pandas dataframe into list of tuples
    consisting of (german sentences, english sentences).
    
    This iterable in used in our data preparation
    """
    sample_iter = sample_df.to_dict(orient='records')
    out_iter = [(dict1["german"], dict1["english"]) for dict1 in sample_iter]
    print(f"length of iterable: {len(out_iter)}")
    return out_iter

In [7]:
train_iter = create_iterable(train_df)
val_iter = create_iterable(val_df)

length of iterable: 27550
length of iterable: 1450


In [8]:
test_df = pd.read_csv("/kaggle/input/machine-translation-dataset-de-en/translation_test.csv")
test_df.head()

,english,german
0,A man in an orange hat starring at something.,"Ein Mann mit einem orangefarbenen Hut, der etw..."
1,A Boston Terrier is running on lush green gras...,Ein Boston Terrier läuft über saftig-grünes Gr...
2,A girl in karate uniform breaking a stick with...,Ein Mädchen in einem Karateanzug bricht einen ...
3,Five people wearing winter jackets and helmets...,Fünf Leute in Winterjacken und mit Helmen steh...
4,People are fixing the roof of a house.,Leute Reparieren das Dach eines Hauses.


In [9]:
test_iter = create_iterable(test_df)

length of iterable: 1000


# Create Vocab from data sources

In [10]:
def load_tokenizers():
    """
    Load the German and English tokenizers provided by spaCy.

    Returns:
        spacy_de:     German tokenizer
        spacy_en:     English tokenizer
    """
    try:
        spacy_de = spacy.load("de_core_news_sm")
    except OSError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except OSError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    print("Loaded English and German tokenizers.")
    return spacy_de, spacy_en

In [11]:
def tokenize(text: str, tokenizer):
  """
    Split a string into its tokens using the provided tokenizer.

    Args:
        text:         string 
        tokenizer:    tokenizer for the language
        
    Returns:
        tokenized list of strings       
  """
  return [tok.text.lower() for tok in tokenizer.tokenizer(text)]

In [12]:
def yield_tokens(data_iter, tokenizer, index: int):
  """
    Return the tokens for the appropriate language.

    Args:
        data_iter:    text here 
        tokenizer:    tokenizer for the language
        index:        index of the language in the tuple | (de=0, en=1)
        
    Yields:
        sequences based on index       
  """
  for from_tuple in data_iter:
    yield tokenizer(from_tuple[index])

In [13]:
def build_vocabulary(
                    spacy_de, 
                    spacy_en, 
                    train_iter, 
                    val_iter, 
                    test_iter, 
                    min_freq: int = 2):
  
    def tokenize_de(text: str):
        """
          Call the German tokenizer.

          Args:
              text:         string 
              min_freq:     minimum frequency needed to include a word in the vocabulary

          Returns:
              tokenized list of strings       
        """
        return tokenize(text, spacy_de)

    def tokenize_en(text: str):
        """
          Call the English tokenizer.

          Args:
              text:         string 

          Returns:
              tokenized list of strings       
        """
        return tokenize(text, spacy_en)

    print("Building German Vocabulary...")

#     # load train, val, and test data pipelines
#     train, val, test = datasets.IWSLT2016(language_pair=("de", "en"))
    train = train_iter
    val = val_iter
    test = test_iter

    # generate source vocabulary
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0), # tokens for each German sentence (index 0)
        min_freq=min_freq, 
        specials=["<bos>", "<eos>", "<pad>", "<unk>"],
    )

    print("Building English Vocabulary...")

    # generate target vocabulary
    vocab_trg = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1), # tokens for each English sentence (index 1)
        min_freq=2, # 
        specials=["<bos>", "<eos>", "<pad>", "<unk>"],
    )

    # set default token for out-of-vocabulary words (OOV)
    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_trg.set_default_index(vocab_trg["<unk>"])

    return vocab_src, vocab_trg

In [14]:
def load_vocab(spacy_de, spacy_en, train_iter, val_iter, test_iter, min_freq: int = 2):
    """
    Args:
        spacy_de:     German tokenizer
        spacy_en:     English tokenizer
        min_freq:     minimum frequency needed to include a word in the vocabulary

    Returns:
        vocab_src:    German vocabulary
        vocab_trg:     English vocabulary       
    """
    if not os.path.exists("vocab.pt"):
        # build the German/English vocabulary if it does not exist
        vocab_src, vocab_trg = build_vocabulary(spacy_de, 
                                                spacy_en, 
                                                train_iter, 
                                                val_iter, 
                                                test_iter, 
                                                min_freq)
        # save it to a file
        torch.save((vocab_src, vocab_trg), "vocab.pt")
    else:
        # load the vocab if it exists
        vocab_src, vocab_trg = torch.load("vocab.pt")

    print("Finished.\nVocabulary sizes:")
    print("\tSource:", len(vocab_src))
    print("\tTarget:", len(vocab_trg))
    return vocab_src, vocab_trg

In [15]:
# global variables used later in the script
spacy_de, spacy_en = load_tokenizers()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 74.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Loaded English and German tokenizers.


In [16]:
vocab_src, vocab_trg = load_vocab(
                            spacy_de, 
                            spacy_en, 
                            train_iter, 
                            val_iter, 
                            test_iter)

Building German Vocabulary...
Building English Vocabulary...
Finished.
Vocabulary sizes:
	Source: 7983
	Target: 5979


In [17]:
BOS_IDX = vocab_trg['<bos>']
EOS_IDX = vocab_trg['<eos>']
PAD_IDX = vocab_trg['<pad>']

# Perform data preprocess

In [18]:
def data_process(raw_data):
    """
    Process raw sentences by tokenizing and converting to integers based on 
    the vocabulary.

    Args:
        raw_data:     German-English sentence pairs 
    Returns:
        data:         tokenized data converted to index based on vocabulary   
    """
    data = []
    # loop through each sentence pair
    for (raw_de, raw_en) in tqdm(raw_data):
        de_tensor_ = []
        # tokenize the sentence and convert each word to an integers
        for token in spacy_de.tokenizer(raw_de):
            de_tensor_.append(vocab_src[token.text.lower()])
            
        en_tensor_ = []
        # tokenize the sentence and convert each word to an integers
        for token in spacy_en.tokenizer(raw_en):
            en_tensor_.append(vocab_trg[token.text.lower()])
            
        de_tensor_ = torch.tensor(de_tensor_, dtype=torch.long)
        en_tensor_ = torch.tensor(en_tensor_, dtype=torch.long)
        # append tensor representations
        data.append((de_tensor_, en_tensor_))
    return data

In [19]:
# processed data
train_data = data_process(train_iter)
print(f"Train data shape: {len(train_data)}")
val_data = data_process(val_iter)
print(f"Val data shape: {len(val_data)}")
test_data = data_process(test_iter)
print(f"Test data shape: {len(test_data)}")

100%|██████████| 27550/27550 [00:03<00:00, 7620.49it/s]


Train data shape: 27550


100%|██████████| 1450/1450 [00:00<00:00, 3744.30it/s]


Val data shape: 1450


100%|██████████| 1000/1000 [00:00<00:00, 7572.37it/s]

Test data shape: 1000


# Create collate function

In [20]:
def generate_batch(data_batch):
    """
    Process indexed-sequences by adding <bos>, <eos>, and <pad> tokens.

    Args:
        data_batch:     German-English indexed-sentence pairs

    Returns:
        two batches:    one for German and one for English
    """
    de_batch, en_batch = [], []

    # for each sentence
    for (de_item, en_item) in data_batch:
        # add <bos> and <eos> indices before and after the sentence
        de_temp = torch.cat([torch.tensor([BOS_IDX]), 
                             de_item, 
                             torch.tensor([EOS_IDX])], dim=0).to(device)
        en_temp = torch.cat([torch.tensor([BOS_IDX]), 
                             en_item, 
                             torch.tensor([EOS_IDX])], dim=0).to(device)

        # add padding
        de_batch.append(pad(de_temp,(0, # dimension to pad
                                MAX_PADDING - len(de_temp), # amount of padding to add
                              ),value=PAD_IDX,))

        # add padding
        en_batch.append(pad(en_temp,(0, # dimension to pad
                                MAX_PADDING - len(en_temp), # amount of padding to add
                              ),
                              value=PAD_IDX,))

    return torch.stack(de_batch), torch.stack(en_batch)

# Creating DataLoaders for training and testing

In [21]:
MAX_PADDING = 20
BATCH_SIZE = 128

train_iter = DataLoader(
                to_map_style_dataset(train_data), 
                batch_size=BATCH_SIZE,
                shuffle=True, 
                drop_last=True, 
                collate_fn=generate_batch)

valid_iter = DataLoader(
                to_map_style_dataset(val_data),
                batch_size=BATCH_SIZE,
                shuffle=True, 
                drop_last=True, 
                collate_fn=generate_batch)

test_iter = DataLoader(
                to_map_style_dataset(test_data), 
                batch_size=BATCH_SIZE,
                shuffle=True, 
                drop_last=True, 
                collate_fn=generate_batch)


# Creating Attention Network Model

In [22]:
import math

## Create submodules for network

In [23]:
class Embedding(nn.Module):
    """
    Embedding lookup table which is used by the positional embedding block.
    Embedding lookup table is shared across input and output
    """
    def __init__(self, vocab_size, dmodel):
        """
        Embedding lookup needs a vocab size and model dimension size matrix for 
        creating lookups
        """
        super().__init__()
        self.embedding_lookup = nn.Embedding(vocab_size, dmodel)
        self.vocab_size = vocab_size
        self.dmodel = dmodel
        
    def forward(self, token_ids):
        """
        For a given token lookup the embedding vector
        
        As per the paper, we also multiply the embedding vector with sqrt of dmodel 
        """
        assert token_ids.ndim == 2, \
        f'Expected: (batch size, max token sequence length), got {token_ids.shape}'
        
        embedding_vector = self.embedding_lookup(token_ids)
        
        return embedding_vector * math.sqrt(self.dmodel)


In [24]:
class PositionalEncoding(nn.Module):
    def __init__(self, dmodel, max_seq_length = 5000, pdropout = 0.1,):
        """
        dmodel(int): model dimensions
        max_seq_length(int): Maximum input sequence length
        pdropout(float): Dropout probability
        """
        super().__init__()
        self.dropout = nn.Dropout(p = pdropout)
        
        # Calculate frequencies
        position_ids = torch.arange(0, max_seq_length).unsqueeze(1)
        # -ve sign is added because the exponents are inverted when you multiply position and frequencies
        frequencies = torch.pow(10000, -torch.arange(0, dmodel, 2, dtype = torch.float)/ dmodel) 
        
        # Create positional encoding table
        positional_encoding_table = torch.zeros(max_seq_length, dmodel)
        # Fill the table with even entries with sin and odd entries with cosine
        positional_encoding_table[:, 0::2] = torch.sin(position_ids * frequencies)
        positional_encoding_table[:, 1::2] = torch.cos(position_ids * frequencies)
    
        # Registering the position enconding in state_dict but the its not included 
        # in named parameter as it is not trainable
        self.register_buffer("positional_encoding_table", positional_encoding_table)
    
    def forward(self, embeddings_batch):
        """
        embeddings_batch shape = (batch size, seq_length, dmodel)
        positional_encoding_table shape = (max_seq_length, dmodel)
        """
        assert embeddings_batch.ndim == 3, \
        f"Embeddings batch should have dimension of 3 but got {embeddings_batch.ndim}"
        assert embeddings_batch.size()[-1] == self.positional_encoding_table.size()[-1], \
        f"Embedding batch shape and positional_encoding_table shape should match, expected Embedding batch shape : {embeddings_batch.shape[-1]} while positional_encoding_table shape : {positional_encoding_table[-1]}"
        
        # Get encodings for the given input sequence length
        pos_encodings = self.positional_encoding_table[:embeddings_batch.shape[1]] # Choose only seq_length out of max_seq_length
        
        # Final output 
        out = embeddings_batch + pos_encodings
        out = self.dropout(out)
        return out

In [25]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, dmodel, dff, pdropout = 0.1):
        super().__init__()
        
        self.dropout = nn.Dropout(p = pdropout)
        
        self.W1 = nn.Linear(dmodel, dff)      # Intermediate layer
        self.W2 = nn.Linear(dff, dmodel)    # Output layer
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Perform Feedforward calculation
        
        x shape = (B - batch size, S/T - max token sequence length, D- model dimension).
        """
        out = self.W2(self.relu(self.dropout(self.W1(x))))
        return out

In [26]:
class MultiHeadAttention(nn.Module):
    """
    We can refer to the following blog to understand in depth about the transformer and MHA
    https://medium.com/@hunter-j-phillips/multi-head-attention-7924371d477a
    
    Here we are clubbing all the linear layers together and duplicating the inputs and 
    then performing matrix multiplications
    """
    def __init__(self, dk, dv, h):
        """
        Input Args:
        
        dk(int): Key dimensions used for generating Key weight matrix
        dv(int): Val dimensions used for generating val weight matrix
        h(int) : Number of heads in MHA
        """
        super().__init__()
        assert dk == dv
        self.dk = dk
        self.dv = dv
        self.h = h
        self.dmodel = self.dk * self.h  # model dimension
        
        # Add the params in modulelist as the params in the conv list needs to be tracked
        # wq, wk, wv -> multiple linear weights for the number of heads
        self.WQ = nn.Linear(self.dmodel, self.dmodel) # shape -> (dmodel, dmodel)
        self.WK = nn.Linear(self.dmodel, self.dmodel) # shape -> (dmodel, dmodel)
        self.WV = nn.Linear(self.dmodel, self.dmodel) # shape -> (dmodel, dmodel)
        # Output Weights
        self.WO = nn.Linear(self.h*self.dv, self.dmodel)  # shape -> (dmodel, dmodel)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, query, key, val, mask=None):
        """
        Forward pass for MHA
        
        X has a size of (batch_size, seq_length, d_model)
        Wq, Wk, and Wv have a size of (d_model, d_model)
        
        Perform Scaled Dot Product Attention on multi head attention. 
        
        Notation: B - batch size, S/T - max src/trg token-sequence length
        query shape = (B, S, dmodel)
        key shape = (B, S, dmodel)
        val shape = (B, S, dmodel)
        """
        # Weight the queries
        Q = self.WQ(query)     # shape -> (B, S, dmodel)
        K = self.WK(key)       # shape -> (B, S, dmodel)
        V = self.WV(val)       # shape -> (B, S, dmodel)
        
        # Separate last dimension to number of head and dk
        batch_size = Q.size(0)   
        Q = Q.view(batch_size, -1, self.h, self.dk)   # shape -> (B, S, h, dk)
        K = K.view(batch_size, -1, self.h, self.dk)   # shape -> (B, S, h, dk)
        V = V.view(batch_size, -1, self.h, self.dk)   # shape -> (B, S, h, dk)
        
        # each sequence is split across n_heads, with each head receiving seq_length tokens 
        # with d_key elements in each token instead of d_model.
        Q = Q.permute(0, 2, 1, 3) # shape -> (B, h, S, dk)
        K = K.permute(0, 2, 1, 3) # shape -> (B, h, S, dk)
        V = V.permute(0, 2, 1, 3) # shape -> (B, h, S, dk)
        
        # dot product of Q and K
        scaled_dot_product = torch.matmul(Q, K.permute(0, 1, 3, 2)) / math.sqrt(self.dk)
        
        # fill those positions of product as (-1e10) where mask positions are 0
        if mask is not None:
            scaled_dot_product = scaled_dot_product.masked_fill(mask == 0, -1e10)
            
        scaled_dot_product = self.softmax(scaled_dot_product)
        attention_prob = scaled_dot_product
        
        # Create head 
        head = torch.matmul(scaled_dot_product, V)  # shape -> (B, h, S, S) * (B, h, S, dk) = (B, h, S, dk)
        # Prepare the head to pass it through output linear layer
        head = head.permute(0, 2, 1, 3).contiguous()  # shape -> (B, S, h, dk)
        # Concatenate the head together
        head = head.view(batch_size, -1, self.h* self.dk)  # shape -> (B, S, (h*dk = dmodel))
        # Pass through output layer
        token_representation = self.WO(head)
        return token_representation, attention_prob
    

## Create Encoder for the Network

In [27]:
class EncoderLayer(nn.Module):
    """
    This building block in the encoder layer consists of the following
    1. MultiHead Attention
    2. Sublayer Logic
    3. Positional FeedForward Network
    """
    def __init__(self, dk, dv, h, dim_multiplier = 4, pdropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(dk, dv, h)
        # Reference page 5 chapter 3.2.2 Multi-head attention
        dmodel = dk*h
        # Reference page 5 chapter 3.3 positionwise FeedForward
        dff = dmodel * dim_multiplier
        self.attn_norm = nn.LayerNorm(dmodel)
        self.ff = PositionwiseFeedForward(dmodel, dff, pdropout=pdropout)
        self.ff_norm = nn.LayerNorm(dmodel)
        
        self.dropout = nn.Dropout(p = pdropout)
        
    def forward(self, src_inputs, src_mask=None):
        """
        Forward pass as per page 3 chapter 3.1
        """
        mha_out, attention_wts = self.attention(
                                query = src_inputs, 
                                key = src_inputs, 
                                val = src_inputs, 
                                mask = src_mask)
        
        # Residual connection between input and sublayer output, details: Page 7, Chapter 5.4 "Regularization",
        # Actual paper design is the following
        intermediate_out = self.attn_norm(src_inputs + self.dropout(mha_out))
        
        pff_out = self.ff(intermediate_out)
        
        # Perform Add Norm again
        out = self.ff_norm(intermediate_out + self.dropout(pff_out))
        return out, attention_wts

In [28]:
class Encoder(nn.Module):
    def __init__(self, dk, dv, h, num_encoders, dim_multiplier = 4, pdropout=0.1):
        super().__init__()
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(dk, 
                         dv, 
                         h, 
                         dim_multiplier, 
                         pdropout) for _ in range(num_encoders)
        ])
        
    def forward(self, src_inputs, src_mask = None):
        """
        Input from the Embedding layer
        src_inputs = (B - batch size, S/T - max token sequence length, D- model dimension)
        """
        src_representation = src_inputs
        
        # Forward pass through encoder stack
        for enc in self.encoder_layers:
            src_representation, attention_wts = enc(src_representation, src_mask)
            
        self.attention_wts = attention_wts
        return src_representation

## Create Decoder Layer

In [29]:
class DecoderLayer(nn.Module):
    def __init__(
                self, 
                dk, 
                dv, 
                h,
                dim_multiplier = 4, 
                pdropout = 0.1):
        super().__init__()
        
        # Reference page 5 chapter 3.2.2 Multi-head attention
        dmodel = dk*h
        # Reference page 5 chapter 3.3 positionwise FeedForward
        dff = dmodel * dim_multiplier
        
        # Masked Multi Head Attention
        self.masked_attention = MultiHeadAttention(dk, dv, h)
        self.masked_attn_norm = nn.LayerNorm(dmodel)
        
        # Multi head attention
        self.attention = MultiHeadAttention(dk, dv, h)
        self.attn_norm = nn.LayerNorm(dmodel)
        
        # Add position FeedForward Network
        self.ff = PositionwiseFeedForward(dmodel, dff, pdropout=pdropout)
        self.ff_norm = nn.LayerNorm(dmodel)
        
        self.dropout = nn.Dropout(p = pdropout)
        
    def forward(self, target_inputs, src_inputs, target_mask, src_mask):
        """
        Input from the Embedding layer
        target_inputs = embedded sequences    (batch_size, trg_seq_length, d_model)
        src_inputs = embedded sequences       (batch_size, src_seq_length, d_model)
        target_mask = mask for the sequences  (batch_size, 1, trg_seq_length, trg_seq_length)
        src_mask = mask for the sequences     (batch_size, 1, 1, src_seq_length)
        """
        mmha_out, attention_wts = self.masked_attention(
                                query = target_inputs, 
                                key = target_inputs, 
                                val = target_inputs, 
                                mask = target_mask)
        
        # Residual connection between input and sublayer output, details: Page 7, Chapter 5.4 "Regularization",
        # Actual paper design is the following
        target_inputs = self.masked_attn_norm(target_inputs + self.dropout(mmha_out))
        
        # Inputs to the decoder attention is given as follows
        # query = previous decoder layer
        # key and val = output of encoder
        # mask = src_mask
        # Reference : page 5 chapter 3.2.3 point 1
        mha_out, attention_wts = self.attention(
                                query = target_inputs, 
                                key = src_inputs, 
                                val = src_inputs, 
                                mask = src_mask)
        target_inputs = self.attn_norm(target_inputs + self.dropout(mha_out))
        
        pff_out = self.ff(target_inputs)
        # Perform Add Norm again
        out = self.ff_norm(target_inputs + self.dropout(pff_out))
        return out, attention_wts

In [30]:
class Decoder(nn.Module):
    def __init__(
                self, 
                dk, 
                dv, 
                h, 
                num_decoders, 
                dim_multiplier = 4, 
                pdropout=0.1):
        super().__init__()
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(dk, 
                         dv, 
                         h, 
                         dim_multiplier, 
                         pdropout) for _ in range(num_decoders)
        ])
        
    def forward(self, target_inputs, src_inputs, target_mask, src_mask):
        """
        Input from the Embedding layer
        target_inputs = embedded sequences    (batch_size, trg_seq_length, d_model)
        src_inputs = embedded sequences       (batch_size, src_seq_length, d_model)
        target_mask = mask for the sequences  (batch_size, 1, trg_seq_length, trg_seq_length)
        src_mask = mask for the sequences     (batch_size, 1, 1, src_seq_length)
        """
        target_representation = target_inputs
        
        # Forward pass through decoder stack
        for layer in self.decoder_layers:
            target_representation, attention_wts = layer(
                                    target_representation,
                                    src_inputs, 
                                    target_mask,
                                    src_mask)
        return target_representation

## Adding all up to construct the complete model for language translation

In [31]:
class Transformer(nn.Module):
    def __init__(self,
                dk, 
                dv, 
                h,
                src_vocab_size,
                target_vocab_size,
                num_encoders,
                num_decoders,
                src_pad_idx,
                target_pad_idx,
                dim_multiplier = 4, 
                pdropout=0.1,
                device = "cpu"
                ):
        super().__init__()
        
        # Reference page 5 chapter 3.2.2 Multi-head attention
        dmodel = dk*h
        # Modules required to build Encoder
        self.src_embeddings = Embedding(src_vocab_size, dmodel)
        self.src_positional_encoding = PositionalEncoding(
                                        dmodel,
                                        max_seq_length = src_vocab_size,
                                        pdropout = pdropout
                                        )
        self.encoder = Encoder(
                                dk, 
                                dv, 
                                h, 
                                num_encoders, 
                                dim_multiplier=dim_multiplier, 
                                pdropout=pdropout)
        
        # Modules required to build Decoder
        self.target_embeddings = Embedding(target_vocab_size, dmodel)
        self.target_positional_encoding = PositionalEncoding(
                                        dmodel,
                                        max_seq_length = target_vocab_size,
                                        pdropout = pdropout
                                        )
        self.decoder = Decoder(
                                dk, 
                                dv, 
                                h, 
                                num_decoders,  
                                dim_multiplier=4, 
                                pdropout=0.1)
        
        # Final output 
        self.linear = nn.Linear(dmodel, target_vocab_size)
        self.softmax = nn.Softmax(dim=-1)
        self.device = device
        self.src_pad_idx = src_pad_idx
        self.target_pad_idx = target_pad_idx
        self.init_params()  
        
    # This part wasn't mentioned in the paper, but it's super important!
    def init_params(self):
        """
        xavier has tremendous impact! I didn't expect
        that the model's perf, with normalization layers, 
        is so dependent on the choice of weight initialization.
        """
        for name, p in self.named_parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
    def make_src_mask(self, src):
        """
        Args:
            src: raw sequences with padding        (batch_size, seq_length) 
            src_pad_idx(int): index where the token need not be attended

        Returns:
            src_mask: mask for each sequence            (batch_size, 1, 1, seq_length)
        """
        batch_size = src.shape[0]
        # assign 1 to tokens that need attended to and 0 to padding tokens, 
        # then add 2 dimensions
        src_mask = (src != self.src_pad_idx).view(batch_size, 1, 1, -1)
        return src_mask
    
    def make_target_mask(self, target):
        """
        Args:
            target:  raw sequences with padding        (batch_size, seq_length)     
            target_pad_idx(int): index where the token need not be attended

        Returns:
            target_mask: mask for each sequence   (batch_size, 1, seq_length, seq_length)
        """

        seq_length = target.shape[1]
        batch_size = target.shape[0]
        
        # assign True to tokens that need attended to and 
        # False to padding tokens, then add 2 dimensions
        target_mask = (target != self.target_pad_idx).view(batch_size, 1, 1, -1) # (batch_size, 1, 1, seq_length)

        # generate subsequent mask
        trg_sub_mask = torch.tril(torch.ones((seq_length, seq_length), device=self.device)).bool() # (batch_size, 1, seq_length, seq_length)

        # bitwise "and" operator | 0 & 0 = 0, 1 & 1 = 1, 1 & 0 = 0
        target_mask = target_mask & trg_sub_mask

        return target_mask
    
    def forward(
        self, 
        src_token_ids_batch, 
        target_token_ids_batch):
        
        # create source and target masks     
        src_mask = self.make_src_mask(
                        src_token_ids_batch) # (batch_size, 1, 1, src_seq_length)
        target_mask = self.make_target_mask(
                        target_token_ids_batch) # (batch_size, 1, trg_seq_length, trg_seq_length)

        # Create embeddings
        src_representations = self.src_embeddings(src_token_ids_batch)
        src_representations = self.src_positional_encoding(src_representations)
        
        target_representations = self.target_embeddings(target_token_ids_batch)
        target_representations = self.target_positional_encoding(target_representations)

        # Encode 
        encoded_src = self.encoder(src_representations, src_mask)
        
        # Decode
        decoded_output = self.decoder(
                                target_representations, 
                                encoded_src, 
                                target_mask, 
                                src_mask)
        
        # Post processing
        out = self.linear(decoded_output)
        # Output 
        out = self.softmax(out)
        return out

## Create Model Instance

In [32]:
src_pad_idx = vocab_src.get_stoi()["<pad>"]
target_pad_idx = vocab_trg.get_stoi()["<pad>"]
src_pad_idx, target_pad_idx

(2, 2)

In [33]:
dk = 64
dv = 64
h = 8
src_vocab_size = len(vocab_src)
target_vocab_size = len(vocab_trg)
src_pad_idx = vocab_src.get_stoi()["<pad>"]
target_pad_idx = vocab_trg.get_stoi()["<pad>"]
num_encoders = 6
num_decoders = 6
dim_multiplier = 4
pdropout=0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(
                dk, 
                dv, 
                h,
                src_vocab_size,
                target_vocab_size,
                num_encoders,
                num_decoders,
                dim_multiplier, 
                pdropout,
                device = device)
model.cuda()
print(model)



Transformer(
  (src_embeddings): Embedding(
    (embedding_lookup): Embedding(7983, 512)
  )
  (src_positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (encoder_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): MultiHeadAttention(
          (WQ): Linear(in_features=512, out_features=512, bias=True)
          (WK): Linear(in_features=512, out_features=512, bias=True)
          (WV): Linear(in_features=512, out_features=512, bias=True)
          (WO): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
        )
        (attn_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (ff): PositionwiseFeedForward(
          (dropout): Dropout(p=0.1, inplace=False)
          (W1): Linear(in_features=512, out_features=2048, bias=True)
          (W2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
        )
        (ff_norm):

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 54,354,267 trainable parameters


# Training part

In [35]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [36]:
def train(model, iterator, optimizer, criterion, clip):
    """
    Train the model on the given data.

    Args:
        model:        Transformer model to be trained
        iterator:     data to be trained on
        optimizer:    optimizer for updating parameters
        criterion:    loss function for updating parameters
        clip:         value to help prevent exploding gradients

    Returns:
        loss for the epoch
    """

    # set the model to training mode
    model.train()

    epoch_loss = 0
    
    # loop through each batch in the iterator
    for i, batch in enumerate(iterator):

        # set the source and target batches    
        src,trg = batch
        
#         print("src", src, src.size())
#         print("trg", trg, trg.size())
        
#         print("trg[:,:-1]", trg[:,:-1], trg[:,:-1].size())
        
        # expected output
        expected_output = trg[:,1:]
#         print("expected_output", expected_output, expected_output.size())

        # zero the gradients
        optimizer.zero_grad()

        # logits for each output
        logits = model(src, trg[:,:-1])
    
        # calculate the loss
        loss = criterion(logits.contiguous().view(-1, logits.shape[-1]), 
                        expected_output.contiguous().view(-1))

        # backpropagation
        loss.backward()

        # clip the weights
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update the weights
        optimizer.step()

        # update the loss
        epoch_loss += loss.item()

    # return the average loss for the epoch
    return epoch_loss / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    """
    Evaluate the model on the given data.

    Args:
        model:        Transformer model to be trained
        iterator:     data to be evaluated
        criterion:    loss function for assessing outputs

    Returns:
        loss for the data
    """

    # set the model to evaluation mode
    model.eval()

    epoch_loss = 0
    
    # evaluate without updating gradients
    with torch.no_grad():

        # loop through each batch in the iterator
        for i, batch in enumerate(iterator):

            # set the source and target batches  
            src, trg = batch
            
            # expected output
            expected_output = trg[:,1:]


            # logits for each output
            logits = model(src, trg[:,:-1])


            # calculate the loss
            loss = criterion(logits.contiguous().view(-1, logits.shape[-1]), 
                          expected_output.contiguous().view(-1))

            # update the loss
            epoch_loss += loss.item()
        
    # return the average loss for the epoch
    return epoch_loss / len(iterator)

In [38]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Perform Training

In [39]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

# loop through each epoch
for epoch in tqdm(range(N_EPOCHS)):
    
    start_time = time.time()

    # calculate the train loss and update the parameters
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    # calculate the loss on the validation set
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    # calculate how long the epoch took
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # save the model when it performs better than the previous run
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

 10%|█         | 1/10 [00:37<05:36, 37.42s/it]

Epoch: 01 | Time: 0m 37s
	Train Loss: 8.577 | Train PPL: 5309.275
	 Val. Loss: 8.572 |  Val. PPL: 5282.067


 20%|██        | 2/10 [01:13<04:55, 36.90s/it]

Epoch: 02 | Time: 0m 36s
	Train Loss: 8.574 | Train PPL: 5292.466
	 Val. Loss: 8.572 |  Val. PPL: 5282.499


 30%|███       | 3/10 [01:51<04:18, 36.97s/it]

...

Epoch: 08 | Time: 0m 36s
	Train Loss: 8.574 | Train PPL: 5292.261
	 Val. Loss: 8.572 |  Val. PPL: 5281.863


 90%|█████████ | 9/10 [05:30<00:36, 36.62s/it]

Epoch: 09 | Time: 0m 36s
	Train Loss: 8.574 | Train PPL: 5292.377
	 Val. Loss: 8.572 |  Val. PPL: 5282.195


100%|██████████| 10/10 [06:07<00:00, 36.71s/it]

Epoch: 10 | Time: 0m 36s
	Train Loss: 8.574 | Train PPL: 5292.378
	 Val. Loss: 8.572 |  Val. PPL: 5282.712


In [40]:
model.load_state_dict(torch.load('transformer-model.pt'))

# calculate the loss on the test set
test_loss = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}')

Test Loss: 8.576 | Test PPL: 5301.291


In [41]:
def translate_sentence(sentence, model, device, max_length = 50):
    """
    Translate a German sentence to its English equivalent.

    Args:
        sentence:     German sentence to be translated to English; list or str
        model:        Transformer model used for translation
        device:       device to perform translation on
        max_length:   maximum token length for translation

    Returns:
        src:                  return the tokenized input
        trg_input:            return the input to the decoder before the final output 
        trg_output:           return the final translation, shifted right
        attn_probs:           return the attention scores for the decoder heads
        masked_attn_probs:    return the masked attention scores for the decoder heads
    """
    
    model.eval()

    # tokenize and index the provided string
    if isinstance(sentence, str):
        src = ['<bos>'] + [token.text.lower() for token in spacy_de(sentence)] +  ['<eos>']
    else:
        src = ['<bos>'] + sentence + ['<eos>']

    # convert to integers
    src_indexes = [vocab_src[token] for token in src]

    # convert list to tensor
    src_tensor = torch.tensor(src_indexes).int().unsqueeze(0).to(device)

    # set <bos> token for target generation
    trg_indexes = [vocab_trg.get_stoi()['<bos>']]

    # generate new tokens
    for i in range(max_length):

        # convert the list to a tensor
        trg_tensor = torch.tensor(trg_indexes).int().unsqueeze(0).to(device)

        # generate the next token 
        with torch.no_grad():

            # generate the logits
            logits = model.forward(src_tensor, trg_tensor)

            # select the newly predicted token
            pred_token = logits.argmax(2)[:,-1].item()

            # if <eos> token or max length, stop generating
            if pred_token == vocab_trg.get_stoi()['<eos>'] or i == (max_length-1):

                # decoder input
                trg_input = vocab_trg.lookup_tokens(trg_indexes)

                # decoder output
                trg_output = vocab_trg.lookup_tokens(logits.argmax(2).squeeze(0).tolist())

                return src, trg_input, trg_output
      
            # else, continue generating
            else:
                # add the token
                trg_indexes.append(pred_token)

In [42]:
# 'a woman with a large purse is walking by a gate'
src = ['eine', 'frau', 'mit', 'einer', 'großen', 'geldbörse', 'geht', 'an', 'einem', 'tor', 'vorbei', '.']

src, trg_input, trg_output = translate_sentence(src, model, device)

print(f'source = {src}')
print(f'target input = {trg_input}')
print(f'target output = {trg_output}')

source = ['<bos>', 'eine', 'frau', 'mit', 'einer', 'großen', 'geldbörse', 'geht', 'an', 'einem', 'tor', 'vorbei', '.', '<eos>']
target input = ['<bos>', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
target output = ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
